#最小二乘拟合模型数据

这是针对物理科学家（例如物理学家，天文学家）或工程师的统计模型的快速介绍。

为什么需要这个？

因为大多数statsmodels是由统计学家编写的，并且它们使用不同的术语，有时甚至是方法，所以很难知道哪些类和函数是相关的以及它们的输入和输出是什么意思。

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

ImportError: cannot import name '_initialization' from 'statsmodels.tsa.statespace' (c:\Users\Administrator\Desktop\StatsModels\statsmodels-docs-zh\statsmodels\tsa\statespace\__init__.py)

## 线性模型


假设您在位置 x 处有关于测量数据点 y 以及测量误差 y_err。

您如何使用statsmodels将直线模型拟合到此数据？

有关广泛的讨论，请参见[Hogg （2010），“数据分析方法：将模型拟合到数据”](https://arxiv.org/abs/1008.4686)  ...我们将使用表1中给出的示例数据。

因此，模型为f（x）= a * x + b，并且在图1上，他们打印了我们想要重现的结果...最佳拟合参数和为此的“标准加权最小二乘拟合”的参数误差 数据是：
* `a = 2.24 +- 0.11`
* `b = 34 +- 18`

In [6]:
data = """
  x   y y_err
201 592    61
244 401    25
 47 583    38
287 402    15
203 495    21
 58 173    15
210 479    27
202 504    14
198 510    30
158 416    16
165 393    14
201 442    25
157 317    52
131 311    16
166 400    34
160 337    31
186 423    42
125 334    26
218 533    16
146 344    22
"""
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
data = pd.read_csv(StringIO(data), delim_whitespace=True).astype(float)

# Note: 对于结果，我们与本文进行了比较，他们删除了前四点
data.head()

NameError: name 'pd' is not defined

要拟合一条直线，请使用加权最小二乘类 [WLS](https://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.WLS.html) 这些参数命名为：
* `exog` = `sm.add_constant(x)`
* `endog` = `y`
* `weights` = `1 / sqrt(y_err)`

Note that `exog` must be a 2-dimensional array with `x` as a column and an extra column of ones. Adding this column of ones means you want to fit the model `y = a * x + b`, leaving it off means you want to fit the model `y = a * x`.

And you have to use the option `cov_type='fixed scale'` to tell `statsmodels` that you really have measurement errors with an absolute scale. If you do not, `statsmodels` will treat the weights as relative weights between the data points and internally re-scale them so that the best-fit model will have `chi**2 / ndf = 1`.

In [0]:
exog = sm.add_constant(data['x'])
endog = data['y']
weights = 1. / (data['y_err'] ** 2)
wls = sm.WLS(endog, exog, weights)
results = wls.fit(cov_type='fixed scale')
print(results.summary())

### Check against scipy.optimize.curve_fit

In [0]:
# 您可以使用 `scipy.optimize.curve_fit` 来获得最合适的参数和参数误差。
from scipy.optimize import curve_fit

def f(x, a, b):
    return a * x + b

xdata = data['x']
ydata = data['y']
p0 = [0, 0] # 初始参数估计
sigma = data['y_err']
popt, pcov = curve_fit(f, xdata, ydata, p0, sigma, absolute_sigma=True)
perr = np.sqrt(np.diag(pcov))
print('a = {0:10.3f} +- {1:10.3f}'.format(popt[0], perr[0]))
print('b = {0:10.3f} +- {1:10.3f}'.format(popt[1], perr[1]))

### Check against self-written cost function

In [0]:
# 你也可以使用 `scipy.optimize.minimize` 甚至编写自己的损失函数。
# 但是，这不会输出参数误差...您必须分别估算 HESSE 矩阵...
from scipy.optimize import minimize

def chi2(pars):
    """Cost function.
    """
    y_model = pars[0] * data['x'] + pars[1]
    chi = (data['y'] - y_model) / data['y_err']
    return np.sum(chi ** 2)

result = minimize(fun=chi2, x0=[0, 0])
popt = result.x
print('a = {0:10.3f}'.format(popt[0]))
print('b = {0:10.3f}'.format(popt[1]))

## 非线性模型

In [0]:
# TODO: 在此我们使用这个示例:http://probfit.readthedocs.org/en/latest/api.html#probfit.costfunc.Chi2Regression